# State Farm Distracted Driver Detection - Resnet50

This notebook contains the absolute final attempt at the Kaggle State Farm Distracted Driver Detection competition using the Resnet50 model. The purpose is to train the best possible model.

## Initial Setup

Import libraries and functions for future use.

In [1]:
# Plots displayed inline in notebook
%matplotlib inline

# Make help libraries available
import sys

sys.path.append('D:/anlaursen/libraries')

# Set visible devices, so as to just use a single GPU.
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [18]:
import numpy as np
import pandas as pd
import gc

from kerastools.resnet50 import Resnet50
from kerastools.utils import get_batches, save_array, load_array, get_classes, do_clip

from keras.preprocessing import image

## Define model

We setup our initial Resnet50 model

In [3]:
resnet = Resnet50()
resnet.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 224, 224, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
___________________________________________________________________________________________

## Setup batches

We define out validation and training badges for modelling

In [4]:
batch_size = 32

#path = ''
path = 'sample/'

train_batches = resnet.get_batches(path + 'train', batch_size = batch_size)
val_batches = resnet.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 1000 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


## Finetune model - Sample

We need to adjust the standard VGG model to our new input with 10 classes, so we finetune it.

In [5]:
resnet.finetune(train_batches)
resnet.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 224, 224, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 230, 230, 3)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        zero_padding2d_1[0][0]           
___________________________________________________________________________________________

We train the model using the default learning rate of 0.001 for a single epoch

In [6]:
resnet.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
31/31 [==============================] - 8s - loss: 2.3033 - acc: 0.1855 - val_loss: 2.1221 - val_acc: 0.2083


We see that the accuracy increases fine on the sample, so we increase the learning rate.

In [7]:
resnet.model.optimizer.lr = 0.1

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 7s - loss: 1.7066 - acc: 0.4547 - val_loss: 2.1257 - val_acc: 0.2941
Epoch 2/4
31/31 [==============================] - 6s - loss: 1.3449 - acc: 0.6452 - val_loss: 1.8576 - val_acc: 0.3235
Epoch 3/4
31/31 [==============================] - 6s - loss: 1.0613 - acc: 0.7649 - val_loss: 1.7060 - val_acc: 0.4265
Epoch 4/4
31/31 [==============================] - 6s - loss: 0.9073 - acc: 0.8245 - val_loss: 1.7678 - val_acc: 0.3235


Try 4 more epochs with lower learning rate.

In [8]:
resnet.model.optimizer.lr = 0.001

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
31/31 [==============================] - 7s - loss: 0.7744 - acc: 0.8600 - val_loss: 1.7194 - val_acc: 0.3824
Epoch 2/4
31/31 [==============================] - 6s - loss: 0.6708 - acc: 0.8941 - val_loss: 1.4540 - val_acc: 0.5147
Epoch 3/4
31/31 [==============================] - 6s - loss: 0.6156 - acc: 0.9104 - val_loss: 1.5116 - val_acc: 0.4706
Epoch 4/4
31/31 [==============================] - 6s - loss: 0.5370 - acc: 0.9304 - val_loss: 1.4187 - val_acc: 0.4853


Seems, this is as far as we can get on the sample data set. A pretty good base line in the area of 0.5 - 0.66.

## Finetune model - Full data

We continue our finetuning on the full data set.

In [9]:
path = ''

train_batches = resnet.get_batches(path + 'train', batch_size = batch_size)
val_batches = resnet.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.


We start with a single epoch

In [12]:
resnet.fit_batch(train_batches, val_batches, 1)

Epoch 1/1
613/613 [==============================] - 243s - loss: 0.4289 - acc: 0.9068 - val_loss: 1.2018 - val_acc: 0.6056


We see huge overfitting problems. We increase the learning rate and see, where that takes us.

In [13]:
resnet.model.optimizer.lr = 0.1

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 151s - loss: 0.2069 - acc: 0.9626 - val_loss: 1.2660 - val_acc: 0.5672
Epoch 2/4
613/613 [==============================] - 149s - loss: 0.1397 - acc: 0.9753 - val_loss: 1.3018 - val_acc: 0.5759
Epoch 3/4
613/613 [==============================] - 150s - loss: 0.1037 - acc: 0.9813 - val_loss: 1.2052 - val_acc: 0.6181
Epoch 4/4
613/613 [==============================] - 150s - loss: 0.0825 - acc: 0.9848 - val_loss: 1.2606 - val_acc: 0.6062


And then we lower the learning rate again, and see where we end up.

In [14]:
resnet.model.optimizer.lr = 0.001

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
613/613 [==============================] - 153s - loss: 0.0655 - acc: 0.9881 - val_loss: 1.4437 - val_acc: 0.5762
Epoch 2/4
613/613 [==============================] - 151s - loss: 0.0535 - acc: 0.9908 - val_loss: 1.3009 - val_acc: 0.6109
Epoch 3/4
613/613 [==============================] - 153s - loss: 0.0461 - acc: 0.9917 - val_loss: 1.4294 - val_acc: 0.6001
Epoch 4/4
613/613 [==============================] - 152s - loss: 0.0391 - acc: 0.9935 - val_loss: 1.5603 - val_acc: 0.5762


So we get near perfect accuracy, but are overfitting quite a lot. Let's save the weights and try a different approach.

In [16]:
resnet.model.save_weights('models/base_resnet50.h5')

## Resnet continued
Let's augment the photos first of all.

In [20]:
gen_t = image.ImageDataGenerator(rotation_range = 15,
                                 height_shift_range = 0.05,
                                 shear_range = 0.1,
                                 channel_shift_range = 20,
                                 width_shift_range = 0.1)
path = ''

train_batches = resnet.get_batches(path + 'train',
                                   gen_t,
                                   batch_size = 44,
                                   shuffle = True,
                                   target_size = (224, 224))
val_batches = resnet.get_batches(path + 'valid', batch_size = batch_size, shuffle = False)

Found 19624 images belonging to 10 classes.
Found 2800 images belonging to 10 classes.


Define a new model, to get new initialisations.

In [22]:
resnet = Resnet50()

Finetune and make all layers trainable

In [24]:
resnet.finetune(train_batches)

layers = resnet.model.layerstrainable = True

Then train the model using the augmented batches. First run some epochs at default learning rate.

In [26]:
resnet.fit_batch(train_batches, val_batches, 4) 

Epoch 1/4
446/446 [==============================] - 358s - loss: 1.3082 - acc: 0.5889 - val_loss: 1.1403 - val_acc: 0.6228
Epoch 2/4
446/446 [==============================] - 348s - loss: 0.7362 - acc: 0.7818 - val_loss: 1.1528 - val_acc: 0.5831
Epoch 3/4
446/446 [==============================] - 348s - loss: 0.5729 - acc: 0.8305 - val_loss: 1.0828 - val_acc: 0.6351
Epoch 4/4
446/446 [==============================] - 347s - loss: 0.4886 - acc: 0.8588 - val_loss: 1.1715 - val_acc: 0.5918


Then increase the learning rate and run some more epochs

In [29]:
resnet.model.optimizer.lr = 0.01

resnet.fit_batch(train_batches, val_batches, 4)

Epoch 1/4
446/446 [==============================] - 362s - loss: 0.4358 - acc: 0.8713 - val_loss: 1.0653 - val_acc: 0.6181
Epoch 2/4
446/446 [==============================] - 351s - loss: 0.3880 - acc: 0.8878 - val_loss: 1.1632 - val_acc: 0.5936
Epoch 3/4
446/446 [==============================] - 352s - loss: 0.3579 - acc: 0.8954 - val_loss: 1.1590 - val_acc: 0.5997
Epoch 4/4
446/446 [==============================] - 351s - loss: 0.3361 - acc: 0.8997 - val_loss: 1.2399 - val_acc: 0.6091


Finally decrease the learning rate and run some more epochs.

In [32]:
resnet.model.optimizer.lr = 0.001

resnet.fit_batch(train_batches, val_batches, 20)

Epoch 1/20
446/446 [==============================] - 407s - loss: 0.1860 - acc: 0.9416 - val_loss: 1.4871 - val_acc: 0.6326
Epoch 2/20
446/446 [==============================] - 363s - loss: 0.1858 - acc: 0.9393 - val_loss: 1.3615 - val_acc: 0.6192
Epoch 3/20
446/446 [==============================] - 358s - loss: 0.1945 - acc: 0.9381 - val_loss: 1.4962 - val_acc: 0.5979
Epoch 4/20
446/446 [==============================] - 357s - loss: 0.1859 - acc: 0.9419 - val_loss: 1.2958 - val_acc: 0.6308
Epoch 5/20
446/446 [==============================] - 363s - loss: 0.1792 - acc: 0.9433 - val_loss: 1.3871 - val_acc: 0.6171
Epoch 6/20
446/446 [==============================] - 366s - loss: 0.1852 - acc: 0.9405 - val_loss: 1.1916 - val_acc: 0.6488
Epoch 7/20
446/446 [==============================] - 369s - loss: 0.1824 - acc: 0.9429 - val_loss: 1.5571 - val_acc: 0.5762
Epoch 8/20
446/446 [==============================] - 377s - loss: 0.1748 - acc: 0.9429 - val_loss: 1.3423 - val_acc: 0.6337


## Pseudolabeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. Afterwards we add the test set as well.

In [22]:
val_pseudo = resnet.predict(val_batches, batch_size = 50)

We concatenate thse pseudo labels with our training labels

In [24]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

And train our model using the extended data set.